In [1]:
from flask import Flask, render_template, url_for, request, redirect
from flask_sqlalchemy import SQLAlchemy
from datetime import date, datetime

from sqlalchemy import except_
#from algorithm import Solve_Problem
import uuid
import pandas as pd
import pyomo.environ as pyo 

In [2]:
import os
cwd = os.getcwd()
cwd

'/Users/kultiginbozdemir/Documents/GitHub/op_room_opt/WebApp'

In [11]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///test.db'
db = SQLAlchemy(app)
op_date_formatted='2022-02-19 00:00:00.000000'

/Users/kultiginbozdemir/opt/anaconda3/envs/CO2/lib/python3.7/site-packages/flask_sqlalchemy/__init__.py:873: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [12]:
db

<SQLAlchemy engine=sqlite:////Users/kultiginbozdemir/Documents/GitHub/op_room_opt/WebApp/test.db>

In [13]:
class User_Entries(db.Model):
    id = db.Column(db.String(200), primary_key=True)
    doctor = db.Column(db.String(200), nullable= False)
    operation_date = db.Column(db.DateTime, nullable=False)
    department_name = db.Column(db.String(200), nullable=False)
    operation_duration  = db.Column(db.Integer, nullable= False)
    operation_urgency = db.Column(db.Integer, nullable=False)
    operation_room = db.Column(db.String(200))
    
class Department_Info(db.Model):
    id = db.Column(db.String(200), primary_key=True)
    department_name = db.Column(db.String(200), nullable=False)
    department_capacity = db.Column(db.Integer, nullable=False)
    date = db.Column(db.DateTime, nullable=False)

class Operation_rooms_Info(db.Model):
    id = db.Column(db.String(200), primary_key=True)
    room_name = db.Column(db.String(200), nullable=False)
    room_capacity = db.Column(db.Integer, nullable=False)
    date = db.Column(db.DateTime, nullable=False)




In [14]:
user_entries = User_Entries.query.filter(User_Entries.operation_date==op_date_formatted).all()
table=[(str(e).split()[1].strip('>'),e.doctor, e.operation_date,e.department_name, e.operation_duration,e.operation_urgency, e.operation_room) for e in user_entries]

basket = pd.DataFrame(table, columns=['id','doctor','operation_date','department_name','operation_duration','operation_urgency','operation_room'])
basket.index=basket['id']
basket

,id,doctor,operation_date,department_name,operation_duration,operation_urgency,operation_room
id,,,,,,,
Department_3-c940a9a2-6e82-4b1b-95d0-0af880b04e3c,Department_3-c940a9a2-6e82-4b1b-95d0-0af880b04e3c,suds,2022-02-19,Department_3,45,3,None
Department_3-1f64e343-8a4f-494d-aa0a-9342a6d20a77,Department_3-1f64e343-8a4f-494d-aa0a-9342a6d20a77,suds,2022-02-19,Department_3,45,3,None
Department_3-3b06048a-bae2-4194-a48a-74afb13a34f9,Department_3-3b06048a-bae2-4194-a48a-74afb13a34f9,suds,2022-02-19,Department_3,45,3,None
Department_3-e3979e96-37e4-49a0-bfc4-e1e8d7704251,Department_3-e3979e96-37e4-49a0-bfc4-e1e8d7704251,suds,2022-02-19,Department_3,45,3,None


In [15]:
departments_info = Department_Info.query.filter(Department_Info.date==op_date_formatted).all()

table2 =[(str(e).split()[1].strip('>'),e.department_name, e.department_capacity,e.date) for e in departments_info]
dep_df=pd.DataFrame(table2, columns=['id','department_name','department_capacity','date'])
dep_df.set_index('id')
dep_df['department_capacity']=dep_df['department_capacity'].astype(int)
dep_df

,id,department_name,department_capacity,date
0,Department_1-819c2d01-a944-4a9e-a7b0-95806145f176,Department_1,90,2022-02-19
1,Department_2-89816045-b059-40ed-b7a4-7abd53a09347,Department_2,120,2022-02-19
2,Department_3-51ef1628-574c-48e8-a5eb-d598cd812555,Department_3,85,2022-02-19
3,Department_4-6b3f9a28-d2d0-4625-831e-a29e426990d8,Department_4,150,2022-02-19
4,Department_5-656adeef-f896-41ba-8021-c71083d30535,Department_5,60,2022-02-19


In [16]:
operation_rooms_info = Operation_rooms_Info.query.filter(Operation_rooms_Info.date==op_date_formatted).all()

table3 =[(str(e).split()[1].strip('>'),e.room_name, e.room_capacity,e.date) for e in operation_rooms_info]
op_df=pd.DataFrame(table3, columns=['id','room_name','room_capacity','date'])
op_df.set_index('id')
op_df['room_capacity']=op_df['room_capacity'].astype(int)
op_df

,id,room_name,room_capacity,date
0,Room_1-7a357496-040a-4ccd-ad98-a2830cd32a4d,Room_1,100,2022-02-19
1,Room_2-a5a303f9-ed03-4218-bcc7-1ec264efe2da,Room_2,450,2022-02-19
2,Room_3-1f66842b-ca38-42d8-9833-63ebfd523a30,Room_3,80,2022-02-19
3,Room_4-751cc5a9-2d34-49ee-a8db-e1ec15dd7954,Room_4,130,2022-02-19
4,Room_5-0982a403-8a10-4ea0-b236-7d1e575a5bef,Room_5,200,2022-02-19


In [17]:
# operations


#departments

# operation rooms


# multi-knapsack, integer divisible

mdl = pyo.ConcreteModel()

# make sure id is also index in database user__entries
mdl.invs = pyo.Set(initialize=list(zip(basket.index, basket["department_name"])))
mdl.bins = pyo.Set(initialize=list(op_df.room_name)) ## list of operations room from db
mdl.deps = pyo.Set(initialize=list(dep_df.department_name)) # list of departments from db

# params
mdl.value   = pyo.Param(mdl.invs, initialize= {(row["id"],row["department_name"]):row["operation_urgency"] for i,row in basket.iterrows()} )
mdl.weight  = pyo.Param(mdl.invs, initialize= {(row["id"],row["department_name"]):row["operation_duration"] for i,row in basket.iterrows()})
mdl.bin_cap = pyo.Param(mdl.bins, initialize= {row["room_name"]:row["room_capacity"] for i,row in op_df.iterrows()} )
mdl.dep_cap = pyo.Param(mdl.deps, initialize= {row["department_name"]:row["department_capacity"] for i,row in dep_df.iterrows()}, mutable=True)



# vars
mdl.X = pyo.Var(mdl.invs, mdl.bins, within=pyo.Binary)     # the amount from invoice i in bin j



### Objective ###

mdl.OBJ = pyo.Objective(expr=sum(mdl.X[i, b]*mdl.value[i] for 
                        i in mdl.invs for
                        b in mdl.bins), sense=pyo.maximize)


### constraints ###

# don't overstuff bin
def bin_limit(self, b):
    return sum(mdl.X[i, b]*mdl.weight[i] for i in mdl.invs) <= mdl.bin_cap[b]
mdl.bin_limit = pyo.Constraint(mdl.bins, rule=bin_limit)

# one_item can be only in a single op room.
def one_item(self, i,d):
    return sum(mdl.X[i,d,b] for b in mdl.bins) <=1
mdl.one_item = pyo.Constraint(mdl.invs, rule=one_item)



# department limits

mdl.dep_limits=pyo.ConstraintList()

for d in mdl.deps:
    d_list=[]
    for i in mdl.X:
        if d==i[1]:
            d_list.append(i)    
    mdl.dep_limits.add(expr=(sum(mdl.X[i]*mdl.weight[i[:2]] for i in d_list)<=mdl.dep_cap[d])) 



# solve it...
solver = pyo.SolverFactory('glpk')
results = solver.solve(mdl)


# save the output into dictionary
dic=dict()

for i in mdl.X:
    if pyo.value(mdl.X[i])==1:
        dic[i[0]]=i[2]


if basket.index.isin(list(dic.keys())).all():
    basket['operation_room'] = basket.index.map(dic) 
    #print(basket)

else:
    print('not possible')

not possible


In [18]:
basket

,id,doctor,operation_date,department_name,operation_duration,operation_urgency,operation_room
id,,,,,,,
Department_3-c940a9a2-6e82-4b1b-95d0-0af880b04e3c,Department_3-c940a9a2-6e82-4b1b-95d0-0af880b04e3c,suds,2022-02-19,Department_3,45,3,None
Department_3-1f64e343-8a4f-494d-aa0a-9342a6d20a77,Department_3-1f64e343-8a4f-494d-aa0a-9342a6d20a77,suds,2022-02-19,Department_3,45,3,None
Department_3-3b06048a-bae2-4194-a48a-74afb13a34f9,Department_3-3b06048a-bae2-4194-a48a-74afb13a34f9,suds,2022-02-19,Department_3,45,3,None
Department_3-e3979e96-37e4-49a0-bfc4-e1e8d7704251,Department_3-e3979e96-37e4-49a0-bfc4-e1e8d7704251,suds,2022-02-19,Department_3,45,3,None


In [23]:
if type(basket)is pd.DataFrame:
    print('yes')

yes
